In [4]:
%matplotlib inline

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<h1> A Machine Learning Model for Solar Flare Predictions </h1>

<h2>I. Overview</h2>

<h3> What is a solar flare? </h3>

A solar flare is a powerful electromagnetic radioactive eruption of the Sun. It occurs in active regions(AR) which are charactirized with magnetic abnormlities - extremely strong and complex magnetic field. A solar flare can release energy of $10^{21}$ to $10^{25}$ J. This leads to disruptions in radio communication technologies and even fallouts. For example, the biggest solar flare ever recorded, happened in 1859 and caused auroras, which could be seen from the Carribean, blackouts and failiure of most telegraph systems over North America and Europe. That is why scientist are trying to find a way to predict when an event like this would happen.